In [1]:
%load_ext lab_black

import pandas as pd

In [2]:
import os
import glob

files = glob.glob(
    os.path.join("/data/data/own_dataset/covid19-zenodo/", "flightlist_20*.csv.gz")
)


def process_csv(i, nrows=None):
    return pd.read_csv(i, nrows=nrows)[
        ["origin", "destination", "typecode", "callsign"]
    ].dropna()


distribution = (
    pd.concat(map(process_csv, files))
    .groupby(["origin", "destination", "typecode"])
    .count()
    .rename(columns={"callsign": "flights"})
    .reset_index()[["origin", "destination", "typecode", "flights"]]
    .sort_values(["flights"], ascending=False)
)
distribution.to_parquet("distribution_raw.parquet")
distribution

/home/luis/.cache/pypoetry/virtualenvs/traffic-0U6Qm8Lw-py3.9/lib/python3.9/site-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


,origin,destination,typecode,flights
2874001,YSSY,YMML,B738,32989
2860607,YMML,YSSY,B738,32961
1508820,KLAS,KU30,EC30,23627
2845126,YBBN,YSSY,B738,21957
2873409,YSSY,YBBN,B738,21877
...,...,...,...,...
1175201,KEND,OK50,TEX2,1
1175199,KEND,OK43,T38,1
1175196,KEND,OK37,BE40,1
1175194,KEND,OK23,BE40,1


In [3]:
distribution = pd.read_parquet("distribution_raw.parquet")
distribution

,origin,destination,typecode,flights
2874001,YSSY,YMML,B738,32989
2860607,YMML,YSSY,B738,32961
1508820,KLAS,KU30,EC30,23627
2845126,YBBN,YSSY,B738,21957
2873409,YSSY,YBBN,B738,21877
...,...,...,...,...
1175201,KEND,OK50,TEX2,1
1175199,KEND,OK43,T38,1
1175196,KEND,OK37,BE40,1
1175194,KEND,OK23,BE40,1


In [4]:
distribution = (
    distribution.merge(
        pd.read_csv("aircraft_type_designators.csv", index_col=[0]),
        how="left",
        left_on="typecode",
        right_on="ac_code_icao",
    )[["ac_code_icao", "origin", "destination", "flights"]]
    .dropna()
    .rename(columns={"ac_code_icao": "typecode"})
)
distribution

,typecode,origin,destination,flights
0,B738,YSSY,YMML,32989
1,B738,YMML,YSSY,32961
3,B738,YBBN,YSSY,21957
4,B738,YSSY,YBBN,21877
7,B738,YMML,YBBN,16559
...,...,...,...,...
2881346,BE40,KEND,OL09,1
2881348,BE40,KEND,OK71,1
2881351,BE40,KEND,OK37,1
2881352,BE40,KEND,OK23,1


In [5]:
distribution = (
    distribution.merge(
        distribution.groupby(["origin", "destination"]).sum().reset_index(),
        on=["origin", "destination"],
        suffixes=("", "_route"),
    )
    .assign(ratio=lambda x: x.flights / x.flights_route)
    .drop(["flights", "flights_route"], axis=1)
)
distribution.to_parquet("distribution.parquet")
distribution

,typecode,origin,destination,ratio
0,B738,YSSY,YMML,0.648879
1,A320,YSSY,YMML,0.139064
2,A321,YSSY,YMML,0.063493
3,A332,YSSY,YMML,0.037333
4,B734,YSSY,YMML,0.018981
...,...,...,...,...
1002964,BE40,KEND,OL09,1.000000
1002965,BE40,KEND,OK71,1.000000
1002966,BE40,KEND,OK37,1.000000
1002967,BE40,KEND,OK23,1.000000
